# Converting leaderboard databases to pandas dataframe

In [1]:
import polars as pl
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import Markdown

In [2]:
pl.__version__

'0.20.10'

In [3]:
# Define the struct dtype for the materials array
materials_struct_dtype = pl.Struct({
    "fulfilled": pl.Int64,
    "required": pl.Int64,
    "tradeSymbol": pl.Utf8,
})

# Define the overall struct dtype for the JSON
json_struct_dtype = pl.Struct({
    "isComplete": pl.Boolean,
    "materials": pl.List(materials_struct_dtype),
    "symbol": pl.Utf8,
}
)

In [4]:
def read_dataframe(reset_date):
    connection_string = f"sqlite:///Users/florian_witteler/programming/spacetraders/flwi-spacetraders/data/database/flwi-spacetraders-leaderboard-reset_{reset_date}.db"
    df = pl.read_database_uri(query="""
    SELECT datetime(timestamp) as ts_string
         , agent_id
         , crawl_run_id
         , credits
         , ship_count
         , construction_json
         , agent_symbol
         , headquarters_waypoint_symbol
         , jump_gate_waypoint_symbol
    from main.crawl_run cr
             join main.leaderboard_entry le on cr.id = le.crawl_run_id
             join main.static_agent_detail sad on le.agent_id = sad.id
    """, uri=connection_string)

    return df \
      .with_columns(pl.col("ts_string").str.to_datetime('%Y-%m-%d %H:%M:%S').alias("ts")) \
      .with_columns(pl.lit(reset_date).alias("reset_date")) \
      .with_columns(pl.col("construction_json").str.json_decode(json_struct_dtype)) \
      .drop("ts_string", "agent_id", "crawl_run_id")

## create dataframe from _all_ sqlite files

In [5]:
df_all = pl.concat([
    read_dataframe("2023_11_18"),
    read_dataframe("2023_12_02"),
    read_dataframe("2023_12_16"),
    read_dataframe("2023_12_30"),
    read_dataframe("2024_01_13"),
    read_dataframe("2024_01_28"),
    read_dataframe("2024_02_11"),
    read_dataframe("2024_02_25"),
    read_dataframe("2024_03_10"),
    read_dataframe("2024_03_24"),
    read_dataframe("2024_04_09"),
    read_dataframe("2024_04_28"),
    read_dataframe("2024_05_19"),
])  

### notes
Roudtrip via parquet is necessary, since we encounter a bug otherwise. Seems to be a problem with concatenation of different dataframes
```text
thread '<unnamed>' panicked at /Users/runner/work/polars/polars/crates/polars-arrow/src/chunk.rs:20:31:
called `Result::unwrap()` on an `Err` value: ComputeError(ErrString("Chunk require all its arrays to have an equal number of rows"))
```

In [6]:
df_all.write_parquet("all_resets.parquet")

In [7]:
df_all = pl.read_parquet("all_resets.parquet")

In [8]:
resets_df = df_all.select(
    pl.col("reset_date").str
    .to_date("%Y_%m_%d")
    .dt
    .strftime('%Y-%m-%d')
    .alias("reset"),
    pl.col("reset_date"),
    pl.col("ts")
).group_by(
    "reset", "reset_date"
).agg(
    pl.col("ts").min().alias("first_ts")
).sort(
    "reset", descending=False
).with_columns(
    reset_id = pl.col("reset").cum_count()
).rename(
    {"reset_date": "reset_date_str"}
)
resets_df.head(5)

reset,reset_date_str,first_ts,reset_id
str,str,datetime[μs],u32
"""2023-11-18""","""2023_11_18""",2023-11-19 12:30:00,1
"""2023-12-02""","""2023_12_02""",2023-12-02 19:10:00,2
"""2023-12-16""","""2023_12_16""",2023-12-16 20:25:00,3
"""2023-12-30""","""2023_12_30""",2023-12-30 17:15:00,4
"""2024-01-13""","""2024_01_13""",2024-01-13 17:25:00,5


In [9]:
construction_site_df = df_all.select(
    pl.col("reset_date")
    .str
    .to_date("%Y_%m_%d")
    .dt
    .strftime('%Y-%m-%d')
    .alias("reset"),
    pl.col("jump_gate_waypoint_symbol")
).unique(
    
).join(
    resets_df, on="reset"
).with_columns(
    id = pl.col("reset").cum_count()
).drop(
    "first_ts"
)
construction_site_df.head(5)

reset,jump_gate_waypoint_symbol,reset_date_str,reset_id,id
str,str,str,u32,u32
"""2023-11-18""","""X1-SU26-I50""","""2023_11_18""",1,1
"""2023-11-18""","""X1-CG50-I57""","""2023_11_18""",1,2
"""2023-11-18""","""X1-RK11-I57""","""2023_11_18""",1,3
"""2023-11-18""","""X1-AJ54-I55""","""2023_11_18""",1,4
"""2023-11-18""","""X1-NX67-I52""","""2023_11_18""",1,5


In [10]:
construction_requirement_df = df_all.group_by(
    pl.col("reset_date")
).agg(
    pl.col("construction_json").last().struct.field("materials").alias("materials")  # there was an error in one of the resets with the advanced circuits which was fixed during the reset
).explode(
    "materials"
).unnest(
    "materials"
).rename(
    {"tradeSymbol": "trade_symbol"}
).join(
    resets_df, left_on="reset_date", right_on="reset_date_str"
).drop(
    "fulfilled", "first_ts"
).with_columns(
    id = pl.col("reset").cum_count()
)

construction_requirement_df.head(5)

reset_date,required,trade_symbol,reset,reset_id,id
str,i64,str,str,u32,u32
"""2024_04_09""",4000,"""FAB_MATS""","""2024-04-09""",11,1
"""2024_04_09""",1200,"""ADVANCED_CIRCU…","""2024-04-09""",11,2
"""2024_04_09""",1,"""QUANTUM_STABIL…","""2024-04-09""",11,3
"""2024_03_10""",4000,"""FAB_MATS""","""2024-03-10""",9,4
"""2024_03_10""",1200,"""ADVANCED_CIRCU…","""2024-03-10""",9,5


In [11]:
static_agent_info_df = df_all.group_by(
    pl.col("reset_date", "agent_symbol")
).agg(
    pl.col("headquarters_waypoint_symbol").first(),
    pl.col("jump_gate_waypoint_symbol").first(),
    pl.col("ts").min(),
).join(
    resets_df, left_on="reset_date", right_on="reset_date_str"
).drop(
    "first_ts"
).join(
    construction_site_df.rename({"id": "construction_site_id"}), left_on=["reset_id", "jump_gate_waypoint_symbol"], right_on=["reset_id", "jump_gate_waypoint_symbol"]
).with_columns(
    id = pl.col("reset").cum_count()
).rename(
    {"ts": "query_time",
    "headquarters_waypoint_symbol": "agent_headquarters_waypoint_symbol"}
).drop(
    "reset_date","reset_date_str", "reset", "reset_right", "first_ts"
)
static_agent_info_df.head(5)

agent_symbol,agent_headquarters_waypoint_symbol,jump_gate_waypoint_symbol,query_time,reset_id,construction_site_id,id
str,str,str,datetime[μs],u32,u32,u32
"""SNAKESINSPACE""","""X1-C6-A1""","""X1-C6-I51""",2024-04-09 15:40:00,11,226,1
"""DAGON""","""X1-UH16-A1""","""X1-UH16-I57""",2024-05-19 21:00:00,13,39,2
"""SG-1-DEVX""","""X1-DY5-A1""","""X1-DY5-I58""",2023-12-02 20:10:00,2,146,3
"""A8RB4R""","""X1-RH52-A1""","""X1-RH52-I56""",2024-02-25 19:05:00,8,123,4
"""TEX443""","""X1-BK66-A1""","""X1-BK66-I53""",2024-01-15 15:05:00,5,13,5


In [12]:
job_run_df = df_all.select(
    pl.col("reset_date"),
    pl.col("ts").alias("query_time")
).unique(
).with_columns(
    id = pl.col("reset_date").cum_count()
).join(
    resets_df, left_on="reset_date", right_on="reset_date_str"
).with_columns(
    event_time_minutes = (pl.col("query_time") - pl.col("first_ts")).dt.total_minutes()
)
job_run_df.head(5)

reset_date,query_time,id,reset,first_ts,reset_id,event_time_minutes
str,datetime[μs],u32,str,datetime[μs],u32,i64
"""2023_11_18""",2023-11-19 14:10:00,1,"""2023-11-18""",2023-11-19 12:30:00,1,100
"""2023_11_18""",2023-11-19 15:50:00,2,"""2023-11-18""",2023-11-19 12:30:00,1,200
"""2023_11_18""",2023-11-19 16:34:59,3,"""2023-11-18""",2023-11-19 12:30:00,1,244
"""2023_11_18""",2023-11-19 16:40:00,4,"""2023-11-18""",2023-11-19 12:30:00,1,250
"""2023_11_18""",2023-11-19 16:55:00,5,"""2023-11-18""",2023-11-19 12:30:00,1,265


In [13]:
agent_log_df = df_all.select(
    "ts", "reset_date", "agent_symbol", "credits", "ship_count", "construction_json"
).join(
    resets_df, left_on="reset_date", right_on="reset_date_str", suffix="_reset"
).drop(
    "reset_date", "reset", "first_ts"
).join(
    static_agent_info_df, on=["reset_id", "agent_symbol"], suffix="_static_agent_info"
).drop(
    "query_time", "first_ts"
).rename(
    {"id": "agent_id",
    "ts": "query_time"}
).join(
    job_run_df, on=["reset_id", "query_time"], suffix="_static_agent_info"
).rename(
    {"id": "job_id"}
)

agent_log_df.head(5)

query_time,agent_symbol,credits,ship_count,construction_json,reset_id,agent_headquarters_waypoint_symbol,jump_gate_waypoint_symbol,construction_site_id,agent_id,reset_date,job_id,reset,first_ts,event_time_minutes
datetime[μs],str,i64,i64,struct[3],u32,str,str,u32,u32,str,u32,str,datetime[μs],i64
2023-11-19 12:30:00,"""XOYCHTE""",175000,2,"{false,[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}],""X1-XZ50-I54""}",1,"""X1-XZ50-A1""","""X1-XZ50-I54""",178,512,"""2023_11_18""",51196,"""2023-11-18""",2023-11-19 12:30:00,0
2023-11-19 12:30:00,"""TVTVTG4""",175000,2,"{false,[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}],""X1-PU57-I59""}",1,"""X1-PU57-A1""","""X1-PU57-I59""",360,449,"""2023_11_18""",51196,"""2023-11-18""",2023-11-19 12:30:00,0
2023-11-19 12:30:00,"""BLABLABLA""",175749,2,"{false,[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}],""X1-FV87-I58""}",1,"""X1-FV87-A1""","""X1-FV87-I58""",361,61,"""2023_11_18""",51196,"""2023-11-18""",2023-11-19 12:30:00,0
2023-11-19 12:30:00,"""BOOZE2""",175000,2,"{false,[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}],""X1-SK36-I63""}",1,"""X1-SK36-A1""","""X1-SK36-I63""",236,60,"""2023_11_18""",51196,"""2023-11-18""",2023-11-19 12:30:00,0
2023-11-19 12:30:00,"""ESEMUSA""",175600,2,"{false,[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}],""X1-XZ50-I54""}",1,"""X1-XZ50-A1""","""X1-XZ50-I54""",178,282,"""2023_11_18""",51196,"""2023-11-18""",2023-11-19 12:30:00,0


In [14]:
construction_details_df = agent_log_df.select(
    pl.col("jump_gate_waypoint_symbol"),
    pl.col("construction_json").struct.field("isComplete").alias("is_complete"),
    pl.col("construction_json").struct.field("materials"),
    pl.col("job_id"),
    pl.col("reset_id")
).join(
    construction_site_df, on=['reset_id', 'jump_gate_waypoint_symbol']
).rename(
    {"id": "construction_site_id"}
)

construction_details_df.head(5)

jump_gate_waypoint_symbol,is_complete,materials,job_id,reset_id,reset,reset_date_str,construction_site_id
str,bool,list[struct[3]],u32,u32,str,str,u32
"""X1-XZ50-I54""",false,"[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}]",51196,1,"""2023-11-18""","""2023_11_18""",178
"""X1-PU57-I59""",false,"[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}]",51196,1,"""2023-11-18""","""2023_11_18""",360
"""X1-FV87-I58""",false,"[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}]",51196,1,"""2023-11-18""","""2023_11_18""",361
"""X1-SK36-I63""",false,"[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}]",51196,1,"""2023-11-18""","""2023_11_18""",236
"""X1-XZ50-I54""",false,"[{0,6000,""FAB_MATS""}, {500,0,""ADVANCED_CIRCUITRY""}, {1,1,""QUANTUM_STABILIZERS""}]",51196,1,"""2023-11-18""","""2023_11_18""",178


In [15]:
construction_log_df = construction_details_df.drop(
    "materials"
).unique(
    
).with_columns(
    id = pl.col("reset").cum_count()
)
construction_log_df.head(5)

jump_gate_waypoint_symbol,is_complete,job_id,reset_id,reset,reset_date_str,construction_site_id,id
str,bool,u32,u32,str,str,u32,u32
"""X1-PU57-I59""",false,51196,1,"""2023-11-18""","""2023_11_18""",360,1
"""X1-SU26-I50""",false,51196,1,"""2023-11-18""","""2023_11_18""",1,2
"""X1-XD26-I60""",false,17195,1,"""2023-11-18""","""2023_11_18""",141,3
"""X1-JS84-I55""",false,17196,1,"""2023-11-18""","""2023_11_18""",102,4
"""X1-UJ41-I59""",false,17196,1,"""2023-11-18""","""2023_11_18""",395,5


In [16]:
construction_material_log_df = construction_details_df.explode(
    "materials"
).unnest(
    "materials"
).unique(
).join(
    construction_log_df, on=["construction_site_id", "job_id"]
).rename(
    {"id": "construction_log_id", 
    "tradeSymbol": "trade_symbol"}
).join(
    construction_requirement_df, on=["reset_id", "trade_symbol"], suffix="_construction_requirement"
).rename(
    {"id": "construction_requirement_id"}
)
construction_material_log_df.head(5)

jump_gate_waypoint_symbol,is_complete,fulfilled,required,trade_symbol,job_id,reset_id,reset,reset_date_str,construction_site_id,jump_gate_waypoint_symbol_right,is_complete_right,reset_id_right,reset_right,reset_date_str_right,construction_log_id,reset_date,required_construction_requirement,reset_construction_requirement,construction_requirement_id
str,bool,i64,i64,str,u32,u32,str,str,u32,str,bool,u32,str,str,u32,str,i64,str,u32
"""X1-XZ50-I54""",false,500,0,"""ADVANCED_CIRCU…",51196,1,"""2023-11-18""","""2023_11_18""",178,"""X1-XZ50-I54""",false,1,"""2023-11-18""","""2023_11_18""",1422974,"""2023_11_18""",500,"""2023-11-18""",26
"""X1-AJ54-I55""",false,0,6000,"""FAB_MATS""",51196,1,"""2023-11-18""","""2023_11_18""",4,"""X1-AJ54-I55""",false,1,"""2023-11-18""","""2023_11_18""",712271,"""2023_11_18""",6000,"""2023-11-18""",25
"""X1-PR4-I57""",false,1,1,"""QUANTUM_STABIL…",51196,1,"""2023-11-18""","""2023_11_18""",140,"""X1-PR4-I57""",false,1,"""2023-11-18""","""2023_11_18""",712272,"""2023_11_18""",1,"""2023-11-18""",27
"""X1-JS84-I55""",false,1,1,"""QUANTUM_STABIL…",51196,1,"""2023-11-18""","""2023_11_18""",102,"""X1-JS84-I55""",false,1,"""2023-11-18""","""2023_11_18""",1067518,"""2023_11_18""",1,"""2023-11-18""",27
"""X1-NB26-I59""",false,1,1,"""QUANTUM_STABIL…",17195,1,"""2023-11-18""","""2023_11_18""",287,"""X1-NB26-I59""",false,1,"""2023-11-18""","""2023_11_18""",889912,"""2023_11_18""",1,"""2023-11-18""",27


In [20]:
db_path = "/Users/florian_witteler/programming/rust/flwi-spacetraders-rust-leaderboard/data/flwi-leaderboard.db"           
db_connection_string = f"sqlite:///{db_path}"

In [21]:
resets_df.drop("reset_date_str").write_database(table_name="reset_date", connection=db_connection_string, if_table_exists='append')

13

In [22]:
construction_site_df.drop("reset", "reset_date_str", "first_ts").write_database(table_name="construction_site", connection=db_connection_string, if_table_exists='append')

432

In [23]:
construction_requirement_df.drop("reset", "reset_date", "first_ts").write_database(table_name="construction_requirement", connection=db_connection_string, if_table_exists='append')

39

In [24]:
static_agent_info_df.drop(
    "reset_date_str", "jump_gate_waypoint_symbol", "first_ts"
).with_columns(
    starting_faction = pl.lit("")
).write_database(
    table_name="static_agent_info", connection=db_connection_string, if_table_exists='append'
)

676

In [25]:
job_run_df.drop("reset_date", "reset", "first_ts").write_database(table_name="job_run", connection=db_connection_string, if_table_exists='append')

56861

In [26]:
agent_log_df.select(
"agent_id",
"job_id",
"credits",
"ship_count"
).write_database(table_name="agent_log", connection=db_connection_string, if_table_exists='append')

2782651

In [27]:
construction_log_df.drop("reset_date_str", "reset", "jump_gate_waypoint_symbol", "reset_id").write_database(table_name="construction_log", connection=db_connection_string, if_table_exists='append')

1778395

In [28]:
construction_material_log_df.select("construction_log_id", "construction_requirement_id", "fulfilled").write_database(table_name="construction_material_log", connection=db_connection_string, if_table_exists='append')

5335281

## execute `ANALYZE` to make use of all the indices

In [29]:
import sqlite3
con = sqlite3.connect(db_path)
cur = con.cursor()
cur.execute("ANALYZE")
con.close()